In [1]:
"""This is the data cleaner program.

This program (created to run in notebooks), transforms (cleans and normalizes)
the outputs from web_scrapper.ipynb. It also matches the data with researchers
from IFMP, thus it relies on that data being up to date.

Created on Sun Aug  1 10:43:34 2021
"""

__version__ = '0.5'
__author__ = 'Juan Ignacio Rodríguez Vinçon'

############################ IMPORT LIBRARIES ################################
import unicodedata
import json
import re
from ast import literal_eval

import numpy as np
import pandas as pd

In [2]:
# anii.json has different length for each attribute. Before making it a 
# Data Frame it needs to have the same length.
with open(r'data_cleaner_inputs\jsons\anii.json', 'r') as fp:       
    anii_dict = json.load(fp)
anii_dict['anio'].extend('n/a')

# Load the data and create a Data Frame from it:
anii = pd.DataFrame(anii_dict)
csic = pd.read_json(r'data_cleaner_inputs\jsons\csic.json')
ei = pd.read_json(r'data_cleaner_inputs\jsons\ei.json')
sni = pd.read_json(r'data_cleaner_inputs\jsons\sni.json')

# Open manually compiled data and data with researchers' names (includes alts.)
manual_df = pd.read_excel(r'data_cleaner_inputs\manually_retrieved_data.xlsx',
                          na_values = 'n/a')
researchers_df = pd.read_excel(r'data_cleaner_inputs\ifmp_researchers.xlsx',
                               na_values = 'n/a')

############################# ANII DATAFRAME ##################################
# Identify missing values.
anii = anii.replace('n/a', np.nan)
# Get the correct year for each project.
anii['anio'] = anii['codigo'].replace({'(.*)((?<=\_)\d{4}(?=\_))(.*)': '\\2'},
                                      regex = True)

# The column "beneficiario" has both the institution and the name of the
# reponsible person for the project. Thus that information must be separated.
anii['institucion'] = (
    anii['beneficiario'].replace(r'^.*\:', '', regex = True).str.strip()
    )
anii['beneficiario'] = (
    anii['beneficiario'].replace(r'\:.*$', '', regex = True).str.strip()
    )
anii = anii.replace(r'^\s*$', np.nan, regex = True)

# Calculate the finishing date for each project:
# Replace non date values with missing values.
anii['fecha de inicio'] = (
    anii['fecha de inicio'].replace('00.00.0000', np.nan)
    )
# Convert 'fecha de inicio' to datetime format
anii['fecha de inicio'] = (
    pd.to_datetime(anii['fecha de inicio'], format = '%d.%m.%Y')
    )
# Assign a variable to store the data in 'duracion' as integers
anii['offset'] = (
    pd.to_numeric(
        anii['duracion'].replace(r'\s\w*$', '', regex = True).str.strip(),
        errors = 'coerce'
        ).fillna(0).astype('Int8')
    )
# Create 'fecha fin' column that adds 'fecha de inicio' and 'offset'
anii['fecha fin'] = (
    anii.apply(
        lambda x: x['fecha de inicio'] + pd.DateOffset(months = x['offset']),
        axis = 1
        )
    )
anii = anii.drop(columns = 'offset')
# Clean the 'responsable' column of multiple whitespaces.
anii['beneficiario'] = (
    anii['beneficiario'].str.replace(' +', ' ', regex = False)
    )
anii['beneficiario'] = anii['beneficiario'].str.split(',')
# Change dtype for 'anio' to integer (Int32).
anii['anio'] = anii['anio'].astype('float').astype('Int32')

############################# SNI DATAFRAME ##################################
# Identify missing values.
sni = sni.replace('n/a', np.nan)
# The 'name' columns is in "Last Name , First Name" format. In order to join
# the datasets it should be in First Name Last Name (or Full Name) format.
sni['nombres'] = (
    sni['nombre'].replace(r'^.*\,', '', regex = True).str.strip() # limpiar con unicode data (tildes)
    )
sni['nombres'] = sni['nombres'].str.replace(' +', ' ', regex = False)
sni['apellidos'] = (
    sni['nombre'].replace(r'\,.*$', '', regex = True).str.strip()
    )
sni['apellidos'] = sni['apellidos'].str.replace(' +', ' ', regex = False)
sni['nombre'] = sni['nombres'] + ' ' + sni['apellidos']

############################## EI DATAFRAME ###################################
# Identify missing values.
ei = ei.replace('n/a', np.nan)
# Create 'fecha de inicio' and 'fecha fin' from 'periodo'. Then convert them
# to datetime format.
ei['periodo'] = ei['periodo'].replace(r'\n', '', regex = True)
ei['fecha de inicio'] = (
    ei['periodo'].replace(r'\-\s.*$', '', regex = True).str.strip()
    )
ei['fecha fin'] = (
    ei['periodo'].replace(r'^.*\s\-', '', regex = True).str.strip()
    )
ei['fecha de inicio'] = (
    pd.to_datetime(ei['fecha de inicio'], format = '%Y-%m-%d')
    )
ei['fecha fin'] = pd.to_datetime(ei['fecha fin'], format = '%Y-%m-%d')
# Extract year from 'convocatoria' and thus create the column 'anio'.
ei['anio'] = (
    ei['convocatoria'].replace(r'^\D*|\s\(.*\)$|\s\-\s.*$',
                               '', regex = True).str.strip()
    )
# Clean 'convocatoria' from having each row start with the word 'convocatoria'
# (thus avoiding redundancy).
ei['convocatoria'] = (
    ei['convocatoria']
    .replace(r'^convocatoria\s\-\s|^convocatoria\sal|^convocatoria\s',
             '', regex = True).str.strip()
    )
# Create column 'enlace' to contain, as a list, 'url_grupo' and 'url_convo'.
ei['enlace'] = ei[['url_grupo', 'url_convo']].values.tolist()
# If dataset was opened (and not in working memory) it may be necessary to
# convert 'responsables' to list (from string) for later use in .explode
#isna = ei_df['responsables'].isna()
#ei_df.loc[isna, 'responsables'] = pd.Series([[]] * isna.sum()).values
ei['responsables'] = ei['responsables'].fillna('[]')
ei['responsables'] = ei['responsables'].apply(literal_eval)
# Change dtype for 'anio' to integer (Int32).
ei['anio'] = ei['anio'].astype('float').astype('Int32')
# Correct mistakes in some values (e.g.: "vazquez" should be "vasquez").
to_repl = (
    ei[ei['grupo'].str.contains('cicea', na = False)].index.to_list()
    )
for i in range(len(to_repl)):
    ei['responsables'].iloc[to_repl[i]].remove('alejandro vazquez')
    ei['responsables'].iloc[to_repl[i]].extend(['alejandro vasquez'])
    
to_repl = (
    ei[ei['grupo'].str.contains('cicada', na = False)].index.to_list()
    )
ei['responsables'].iloc[to_repl[0]].extend(['alvaro cabana fajardo'])

############################ CSIC DATAFRAME ###################################
# Identify missing values.
csic = csic.replace('n/a', np.nan)
# If dataset was opened (and not in working memory) it may be necessary to
# convert 'responsables' to list (from string) for later use in .explode
#isna = csic_df['responsables'].isna()
#csic_df.loc[isna, 'responsables'] = pd.Series([[]] * isna.sum()).values
csic['responsables'] = csic['responsables'].fillna('[]')
csic['responsables'] = csic['responsables'].apply(literal_eval)
# Change dtype for 'anio' to integer (Int32).
csic['ano'] = csic['ano'].astype('float').astype('Int32')

############################ MANUAL DATAFRAME ###################################
manual_df['responsables'] = manual_df['responsables'].apply(literal_eval)
# Change dtype for 'anio' to integer (Int32).
manual_df['anio'] = manual_df['anio'].astype('float').astype('Int32')

############################ RESEARCHERS DF ###################################
# Normalize data in the columns.
for col in researchers_df.columns[:-2]:
    researchers_df[col] = (researchers_df[col]
                           .str.lower()
                           .str.normalize('NFKD')   
                           .str.encode('ascii', 'ignore')
                           .str.decode('utf8')
                           .str.strip())

# Replace common names, that can be matched erronously, with NaN.
to_repl = ['diego gonzalez', # antes estaba "gonzalez garcia"
           'cecilia gomez', # problema empty string, debería estar solucionado
           'ignacio rodriguez', # antes estaba "monica da silva"
           'nicolas lopez', # empty string, debería estar solucionado
           'monica perez',
           'juan rodriguez',
           'sebastian morales',
           'daniel perez'] # agregado "daniel perez", situación empty string
researchers_df = researchers_df.replace(to_repl, np.nan)

# Create a Regex list out of the columns in 'cols' in order to have matches
# when merging dataframes. This will account for name variations, for example,
# it would match 'alejandro maiche' and 'alejandro maiche marini'.
cols = [c for c in researchers_df.columns[5:-2]]    
researchers_df = researchers_df.fillna('')
to_repl = []
for idx, row in researchers_df[cols].iterrows():
    tmp = row.to_list()
    tmp[:] = [x for x in tmp if x]
    to_repl.append('|'.join(tmp))
# Create a new column with the Regex, replace the empty strings with NaN and
# clean the DF of NaN (those that don't have any name variation, e.g.: 'cecilia
# gomez'). Thus avoiding matching with empty strings when "merging".
researchers_df['regex'] = to_repl
researchers_df = researchers_df.replace('', np.nan)
researchers_df = researchers_df.dropna(how = 'all', subset = cols)
to_repl = researchers_df['regex'].to_list()
vals = researchers_df['id'].to_list()
# Drop columns that are not needed in the merge
cols.extend(['grado', 'primer nombre', 'segundo nombre', 
            'primer apellido', 'segundo apellido', 'regex'])
researchers_df = researchers_df.drop(columns = cols)

########################## REARRANGING COLUMNS ################################
# Rename and rearrange columns order.
anii = anii.rename(
    columns = {'beneficiario': 'responsables',
               'instrumento': 'convocatoria',
               'codigo': 'id',
               'fase_estado': 'fase estado anii',
               'subsidio': 'monto',
               'proyecto': 'nombre de proyecto / grupo',
               'institucion': 'instituciones',
               'area': 'area anii',
               'sector': 'area'}
    )
cols = ['id', 'responsables', 'instituciones', 'convocatoria', 'anio',
        'fecha de inicio', 'fecha fin', 'duracion', 'area', 'monto']
anii = anii[cols + [c for c in anii.columns if c not in cols]]

csic = csic.rename(
    columns = {'ano': 'anio',
               'monto total': 'monto',
               'proyecto': 'nombre de proyecto / grupo',
               'programa': 'convocatoria',
               'area proyecto': 'area'}
    )
cols = ['responsables', 'convocatoria', 'anio', 'monto']
csic = csic[cols + [c for c in csic.columns if c not in cols]]

ei = ei.rename(
    columns = {'servicios involucrados': 'instituciones',
               'grupo': 'nombre de proyecto / grupo'}
    )
cols = ['responsables', 'instituciones', 'convocatoria', 'anio',
        'fecha de inicio', 'fecha fin', 'periodo']
ei = ei[cols + [c for c in ei.columns if c not in cols]]

sni = sni.rename(columns = {'nombre': 'nombre completo'})
cols = ['nombre completo', 'nombres', 'apellidos', 'nivel']
sni = sni[cols + [c for c in sni.columns if c not in cols]]

ValueError: malformed node or string: ['alejandro vazquez', 'juan carlos valle lisboa']

In [ ]:
ei

,grupo,convocatoria,periodo,url_grupo,url_convo,descripcion,objetivos,servicios involucrados,responsables,fecha de inicio,fecha fin
0,cicea,convocatoria centros 2020 (ejecucion 2021-2025),2021-12-01\n - 2025-12-31,https://ei.udelar.edu.uy/grupos-financiados/si...,https://ei.udelar.edu.uy/convocatorias-llamado...,se denominan ciencias cognitivas al conjunto d...,objetivos generales el centro interdisciplina...,"[facultad de psicologia, facultad de ciencias,...","[alejandro vazquez, juan carlos valle lisboa]",2021-12-01,2021-12-01 - 2025-12-31
1,ceifem,convocatoria centros 2020 (ejecucion 2021-2025),2021-01-01\n - 2025-12-31,https://ei.udelar.edu.uy/grupos-financiados/si...,https://ei.udelar.edu.uy/convocatorias-llamado...,la propuesta de creacion del centro de estudio...,consolidar y ampliar los estudios feministas e...,"[facultad de derecho, facultad de informacion ...","[susana rostagnol, niki johnson]",2021-01-01,2021-01-01 - 2025-12-31
2,cicada,convocatoria centros 2020 (ejecucion 2021-2025),2021-01-01\n - 2025-12-31,https://ei.udelar.edu.uy/grupos-financiados/si...,https://ei.udelar.edu.uy/convocatorias-llamado...,esta propuesta aborda la creacion y consolidac...,el objetivo general del centro es la creacion ...,"[facultad de ingenieria, facultad de ciencias,...","[paola bermolen, hector romero]",2021-01-01,2021-01-01 - 2025-12-31
3,ceibos,convocatoria centros 2020 (ejecucion 2021-2025),2021-01-01\n - 2025-12-31,https://ei.udelar.edu.uy/grupos-financiados/si...,https://ei.udelar.edu.uy/convocatorias-llamado...,la biodiversidad de un territorio constituye p...,generales 1) resguardar y valorizar la divers...,"[facultad de quimica, facultad de ciencias, fa...","[sonia rodriguez giordano, pablo dans, carlos ...",2021-01-01,2021-01-01 - 2025-12-31
4,ceinbio,convocatoria centros 2020 (ejecucion 2021-2025),2021-01-01\n - 2025-12-31,https://ei.udelar.edu.uy/grupos-financiados/si...,https://ei.udelar.edu.uy/convocatorias-llamado...,si bien el centro de investigacion tiene su un...,consolidar y proyectar actividades academicas ...,"[facultad de medicina, facultad de ciencias, f...",[rafael radi],2021-01-01,2021-01-01 - 2025-12-31
...,...,...,...,...,...,...,...,...,...,...,...
104,asentamientos irregulares y el acceso a la viv...,convocatoria - semilleros de iniciativas inter...,2015-01-01\n - 2015-08-31,https://ei.udelar.edu.uy/grupos-financiados/si...,https://ei.udelar.edu.uy/convocatorias-llamado...,el acceso a la vivienda y la calidad de la mis...,"para ello se propone como actividad central, l...",[cenur noroeste sede salto],"[jorge rodriguez, natalia bisio]",2015-01-01,2015-01-01 - 2015-08-31
105,"emprendimientos socioproductivos, desmanicomia...",convocatoria - semilleros de iniciativas inter...,2015-01-01\n - 2015-08-31,https://ei.udelar.edu.uy/grupos-financiados/si...,https://ei.udelar.edu.uy/convocatorias-llamado...,se pretende abordar la vinculacion entre los c...,apuntando a contribuir desde una perspectiva i...,[facultad de psicologia],[cecilia baroni],2015-01-01,2015-01-01 - 2015-08-31
106,modulo de ctid,convocatoria - semilleros de iniciativas inter...,2015-01-01\n - 2015-08-31,https://ei.udelar.edu.uy/grupos-financiados/si...,https://ei.udelar.edu.uy/convocatorias-llamado...,la presente propuesta tiene como objetivo el d...,NaN,[comision sectorial de investigacion cientific...,"[reto bertoni, leticia mederos]",2015-01-01,2015-01-01 - 2015-08-31
107,estudios en bioseguridad,convocatoria - semilleros de iniciativas inter...,2015-01-01\n - 2015-08-31,https://ei.udelar.edu.uy/grupos-financiados/si...,https://ei.udelar.edu.uy/convocatorias-llamado...,la bioseguridad constituye un enfoque estrateg...,NaN,[facultad de ciencias],[claudio martinez],2015-01-01,2015-01-01 - 2015-08-31


In [ ]:
########################## PROJECTS MERGING ################################
# Add a column to identify the founding party once the DFs are joined
anii['institucion financiadora'] = 'anii'
csic['institucion financiadora'] = 'csic'
ei['institucion financiadora'] = 'ei'
for col in ['fase estado anii', 'area anii']:
    ei[col] = np.nan
    csic[col] = np.nan

cols =  ['responsables',
         'instituciones',
         'institucion financiadora',
         'convocatoria',
         'anio',
         'fecha de inicio',
         'fecha fin',
         'area',
         'monto',
         'nombre de proyecto / grupo',
         'enlace',
         'fase estado anii',
         'area anii'] # agregar id anii? y dpto anii?

# Drop columns that will not be used in joined DF.
for df in [anii, csic, ei]:
    df.drop(columns = [c for c in df.columns if c not in cols], inplace = True)
# Concat DFs, one with all the projects, another with research only projects.
projects_df  = pd.DataFrame(columns = cols)
#research_df = pd.DataFrame(columns = cols)
projects_df = pd.concat([anii, csic, ei, manual_df], ignore_index = True)
#research_df = pd.concat([anii_df[anii_df['area anii'] == 'investigacion'],
 #                        csic_df, ei_df, manual_df], ignore_index = True)

# Final "merge" that identifies researchers from IFyMP and assigns them an ID. <<<<<<<<<<<<<<<<<<<<< revisar
projects_df = (
    projects_df[projects_df['responsables'].notna()]
    .explode('responsables', True)
    )
projects_df['id'] = projects_df['responsables'].replace(to_repl, vals,
                                                        regex = True)
projects_df['id'] = projects_df['id'].replace({'(.*)(psi_ifmp\S*)(.*)': '\\2'},
                                              regex = True)
projects_df = pd.merge(projects_df, researchers_df, how = 'left', on = 'id')
# Filter by researchers from IFMP and save to Excel.
projects_df = projects_df[projects_df['id'].str.contains('psi', na = False)]
projects_df = projects_df.reset_index(drop = True)
projects_df.to_excel('investigacion.xlsx')
# Filter by most recent (and probably ongoing) research projects only.
flter = (
    ((projects_df['fecha fin'] > '2022-01-01') | 
    (projects_df['fecha fin'].isna())) &
    (projects_df['anio'] > 2017) & 
    ((projects_df['area anii'] == 'investigacion') |
    projects_df['area anii'].isna()) &
    (projects_df['convocatoria'] != 'becas posgrado')
    )
research_df = projects_df[flter].reset_index(drop = True)
research_df.to_excel('proyectos.xlsx')

############################# SNI MERGING #####################################
# "Merge" that identifies researchers from IFyMP in SNI (assigning an ID).
sni['id'] = sni['nombre completo'].replace(to_repl, vals, regex = True)
sni['id'] = sni['id'].replace({'(.*)(psi_ifmp\S*)(.*)': '\\2'},
                              regex = True)
sni = pd.merge(sni, researchers_df, how = 'left', on = 'id')
# Filter by IFyMP researchers, then export to Excel.
ifmp_sni_df = sni[sni['id'].str.contains('psi', na = False)]
ifmp_sni_df = ifmp_sni_df.reset_index(drop = True)
ifmp_sni_df.to_excel('sni_ifmp.xlsx')